In [ ]:
# Selenium

!pip install selenium

In [ ]:
from selenium import webdriver

In [ ]:
!apt-get update

In [ ]:
!apt-get install chromium-driver

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
driver = web_driver()

In [ ]:
from selenium.webdriver.common.by import By
import time
from datetime import date

In [ ]:
# Google Sheets

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('Nome da Planilha que será atualizada').sheet1
rows = worksheet.get_all_values() #atribui os valores da planilha em uma lista de listas à uma variável (onde cada lista é uma linha da planilha)

In [ ]:
# dicionário para armazenar os protocolos já consultados
protocolo_cache = {} #armazenamento de protocolos já consultados

for i in range(1, len(rows)):
    protocolo = rows[i][11]

    if protocolo:

        # verifica se o protocolo já foi consultado
        if protocolo in protocolo_cache:
            # reutiliza os dados já obtidos para o protocolo
            hoje, onde, quando, motivo = protocolo_cache[protocolo]
        else:
            driver.get('URL do site desejado') #abre a página no navegador criado pelo selenium
            while len(driver.find_elements(By.XPATH, '//*[@id="numeroProtocolo"]')) < 1: #aguarda carregamento da página
                time.sleep(0.3)
            time.sleep(0.3)

            pesquisa = driver.find_element(By.XPATH, '//*[@id="numeroProtocolo"]')
            pesquisa.send_keys(protocolo)
            driver.find_element(By.XPATH, '//*[@id="btnPesquisar"]').click()
            while len(driver.find_elements(By.XPATH, '//*[@id="UltimoAndamento_menos"]/div/table/tbody/tr[2]/td[4]')) < 1: #aguarda carregamento da página
                time.sleep(0.3)
            time.sleep(0.3)

            hoje = date.today().strftime('%d/%m/%Y')
            onde = driver.find_element(By.XPATH, '//*[@id="UltimoAndamento_menos"]/div/table/tbody/tr[1]/td[4]').text
            quando = driver.find_element(By.XPATH, '//*[@id="UltimoAndamento_menos"]/div/table/tbody/tr[2]/td[4]').text
            motivo = driver.find_element(By.XPATH, '//*[@id="UltimoAndamento_menos"]/div/table/tbody/tr[2]/td[2]').text

            # armazena o resultado no cache
            protocolo_cache[protocolo] = (hoje, onde, quando, motivo)

        # atualiza as células na planilha com os dados (reutilizados ou novos)
        worksheet.update_acell(f'R{i+1}', hoje)
        worksheet.update_acell(f'O{i+1}', onde)
        worksheet.update_acell(f'P{i+1}', quando)
        worksheet.update_acell(f'Q{i+1}', motivo)